# Downloading GraphAny

In [1]:
!git clone https://github.com/DeepGraphLearning/GraphAny.git

Cloning into 'GraphAny'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 76 (delta 24), reused 39 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (76/76), 576.78 KiB | 6.79 MiB/s, done.
Resolving deltas: 100% (24/24), done.


# Imports and Environment Setup

In [2]:
import sys
import os
import subprocess

sys.path.insert(0,'/content/GraphAny')

!wget -O Miniconda.sh https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda.sh -b -p /usr/local/miniconda

os.environ['PATH'] = '/usr/local/miniconda/bin:' + os.environ['PATH']

!conda update conda -y -q
!source /usr/local/etc/profile.d/conda.sh
!conda init
!conda install -n root _license -y -q

!conda env create -f GraphAny/environment.yaml

Output streaming troncato alle ultime 5000 righe.





libcurand-10.3.5.147 | 51.8 MB   | :   0% 0.00030184652548561843/1 [00:39<36:37:04, 131864.54s/it]












pillow-11.1.0        | 40.5 MB   | :   0% 0.00038623991996082906/1 [00:39<28:39:08, 103188.29s/it]









mkl-2024.2.2         | 118.9 MB  | :  56% 0.5612036480761852/1 [00:39<00:01,  3.89s/it]











libcurand-10.3.5.147 | 51.8 MB   | :   7% 0.07183947306557718/1 [00:39<06:01, 389.75s/it]         












pillow-11.1.0        | 40.5 MB   | :   4% 0.044803830715456176/1 [00:39<09:58, 626.71s/it]        




libcublas-12.4.5.8   | 309.2 MB  | : 100% 1.0/1 [00:40<00:00, 43.82s/it]               













pytorch-2.4.1        | 31.0 MB   | :   0% 0.0005042392381284882/1 [00:40<22:04:32, 79512.62s/it]













pytorch-2.4.1        | 31.0 MB   | :   2% 0.016135655620111624/1 [00:40<28:54, 1762.53s/it]     










libcusolver-11.6.1.9 | 114.0 MB  | :  10% 0.10430889213314093/1 [00:40<02:47, 187.37s/it]







In [3]:
%%bash
source activate graphany

python
import sys
import os
import subprocess
# some simple python commands
sys.path.append('/usr/local/lib/python3.10/site-packages')
print(sys.path)

print("Python version")
print(sys.version)

['', '/env/python', '/usr/local/miniconda/envs/graphany/lib/python310.zip', '/usr/local/miniconda/envs/graphany/lib/python3.10', '/usr/local/miniconda/envs/graphany/lib/python3.10/lib-dynload', '/usr/local/miniconda/envs/graphany/lib/python3.10/site-packages', '/usr/local/lib/python3.10/site-packages']
Python version
3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]


In [4]:
!source activate graphany && conda config --add channels pytorch
!source activate graphany && conda config --add channels pyg
!source activate graphany && conda config --add channels nvidia
!source activate graphany && conda config --add channels conda-forge
!source activate graphany && conda config --add channels dglteam/label/cu118

In [5]:
!source activate graphany && conda install -y pytorch==2.2.1

Channels:
 - dglteam/label/cu118
 - conda-forge
 - nvidia
 - pyg
 - pytorch
 - defaults
Platform: linux-64
Solving environment: \ | / - \ | / - \ | / - \ | / done

## Package Plan ##

  environment location: /usr/local/miniconda/envs/graphany

  added / updated specs:
    - pytorch==2.2.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blas-1.0                   |              mkl           1 KB  conda-forge
    cuda-cudart-11.8.89        |                0         197 KB  nvidia
    cuda-cupti-11.8.87         |                0        25.3 MB  nvidia
    cuda-libraries-11.8.0      |                0           1 KB  nvidia
    cuda-nvrtc-11.8.89         |                0        19.1 MB  nvidia
    cuda-nvtx-11.8.86          |                0          57 KB  nvidia
    cuda-runtime-11.8.0        |                0           1 KB  nvidia
    libblas-3.9.0 

In [6]:
!source activate graphany && conda install -y torchdata==0.7.1

Channels:
 - dglteam/label/cu118
 - conda-forge
 - nvidia
 - pyg
 - pytorch
 - defaults
Platform: linux-64
Solving environment: \ | / - \ | / - \ | / - \ | / done

## Package Plan ##

  environment location: /usr/local/miniconda/envs/graphany

  added / updated specs:
    - torchdata==0.7.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    aws-c-auth-0.7.22          |       h96bc93b_2         103 KB  conda-forge
    aws-c-cal-0.6.14           |       h88a6e22_1          46 KB  conda-forge
    aws-c-common-0.9.19        |       h4ab18f5_0         221 KB  conda-forge
    aws-c-compression-0.2.18   |       h83b837d_6          19 KB  conda-forge
    aws-c-event-stream-0.4.2   |      ha47c788_12          53 KB  conda-forge
    aws-c-http-0.8.1           |      h29d6fba_17         190 KB  conda-forge
    aws-c-io-0.14.8            |       h21d4f22_5         154 KB

In [8]:
!source activate graphany && pip install torch_frame

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 110.1 MB/s eta 0:00:00


In [9]:
!source activate graphany && pip install pytorch-frame[full]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 144.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 114.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 113.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 132.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 146.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 119.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.5/599.5 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 67.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.3
    Uninstalling numpy-2.2.3:
      Successfully uninsta

In [10]:
!source activate graphany && conda install -y tensorboard

Channels:
 - dglteam/label/cu118
 - conda-forge
 - nvidia
 - pyg
 - pytorch
 - defaults
Platform: linux-64
Solving environment: - \ | / - \ | / - \ | done

## Package Plan ##

  environment location: /usr/local/miniconda/envs/graphany

  added / updated specs:
    - tensorboard


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    absl-py-2.1.0              |     pyhd8ed1ab_1         105 KB  conda-forge
    blinker-1.9.0              |     pyhff2d567_0          14 KB  conda-forge
    cachetools-5.5.1           |     pyhd8ed1ab_0          15 KB  conda-forge
    cryptography-44.0.1        |  py310h6c63255_0         1.5 MB  conda-forge
    google-auth-2.38.0         |     pyhd8ed1ab_0         114 KB  conda-forge
    google-auth-oauthlib-0.4.6 |     pyhd8ed1ab_0          19 KB  conda-forge
    grpcio-1.46.3              |  py310hba10ccf_0         2.6 MB  conda-forge
    mar

In [11]:
!source activate graphany && conda list torch

# packages in environment at /usr/local/miniconda/envs/graphany:
#
# Name                    Version                   Build  Channel
pytorch                   2.2.1           py3.10_cuda11.8_cudnn8.7.0_0    pytorch
pytorch-cuda              11.8                 h7e8668a_6    pytorch
pytorch-frame             0.2.5                    pypi_0    pypi
pytorch-lightning         2.5.0.post0        pyh101cb37_0    conda-forge
pytorch-mutex             1.0                        cuda    pytorch
torch-frame               1.7.5                    pypi_0    pypi
torchaudio                2.2.1               py310_cu118    pytorch
torchdata                 0.7.1                     py310    pytorch
torchmetrics              1.6.1              pyhd8ed1ab_0    conda-forge
torchtriton               2.2.0                     py310    pytorch
torchvision               0.17.1              py310_cu118    pytorch


# F1 Dataset Implementation

## Update `configs/data.yaml`

In [12]:
import yaml
# leggiamo tutto il file yaml
file_path = 'GraphAny/configs/data.yaml'

with open(file_path, 'r') as file:
    data = yaml.safe_load(file)

# aggiungo i metadati del dataset F1 al file yaml
data['_ds_meta_data']['F1'] = 'relbench, f1_9_classes3'


with open(file_path, 'w') as file:
    yaml.dump(data, file, default_flow_style=False, sort_keys=False)

In [13]:
with open(file_path, 'r') as file:
    data = yaml.safe_load(file)


# Aggiungo il nuovo elemento _dataset_lookup
data['_dataset_lookup']['F1Debug'] = {
    'train': ['Wisconsin'],
    'eval': ['F1']
}


with open(file_path, 'w') as file:
    yaml.dump(data, file, default_flow_style=False, sort_keys=False)

## Implement the dataset interface and update `GraphDataset` class in `data.py`

In [38]:
new_code = #"""

import logging
import os
import os.path
import os.path as osp
import re
import ssl
import sys
import urllib

import pandas as pd
import pickle
from torch_frame import TensorFrame

import dgl
import dgl.function as fn
import numpy as np
import pytorch_lightning as pl
import torch
from hydra.utils import instantiate
from omegaconf import OmegaConf
from scipy.spatial.distance import pdist, squareform
from sklearn.manifold._utils import (
    _binary_search_perplexity as sklearn_binary_search_perplexity,
)
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

from graphany.utils import logger, timer


def get_entropy_normed_cond_gaussian_prob(X, entropy, metric="euclidean"):

    #Parameters
    #----------
    #X:              The matrix for pairwise similarity
    #entropy:     Perplexity of the conditional prob distribution
    #Returns the entropy-normalized conditional gaussian probability based on distances.
    #-------


    # Compute pairwise distances
    perplexity = np.exp2(entropy)
    distances = pdist(X, metric=metric)
    distances = squareform(distances)

    # Compute the squared distances
    distances **= 2
    distances = distances.astype(np.float32)
    return sklearn_binary_search_perplexity(distances, perplexity, verbose=0)


def sample_k_nodes_per_label(label, visible_nodes, k, num_class):
    ref_node_idx = [
        (label[visible_nodes] == lbl).nonzero().view(-1) for lbl in range(num_class)
    ]
    sampled_indices = [
        label_indices[torch.randperm(len(label_indices))[:k]]
        for label_indices in ref_node_idx
    ]
    return visible_nodes[torch.cat(sampled_indices)]


def get_data_split_masks(n_nodes, labels, num_train_nodes, label_idx=None, seed=42):
    label_idx = np.arange(n_nodes)
    test_rate_in_labeled_nodes = (len(labels) - num_train_nodes) / len(labels)
    train_idx, test_and_valid_idx = train_test_split(
        label_idx,
        test_size=test_rate_in_labeled_nodes,
        random_state=seed,
        shuffle=True,
        stratify=labels,
    )
    valid_idx, test_idx = train_test_split(
        test_and_valid_idx,
        test_size=0.5,
        random_state=seed,
        shuffle=True,
        stratify=labels[test_and_valid_idx],
    )
    train_mask = torch.zeros(n_nodes, dtype=torch.bool)
    val_mask = torch.zeros(n_nodes, dtype=torch.bool)
    test_mask = torch.zeros(n_nodes, dtype=torch.bool)

    train_mask[train_idx] = True
    val_mask[valid_idx] = True
    test_mask[test_idx] = True

    return train_mask, val_mask, test_mask


def download_url(url: str, folder: str, log: bool = True, filename=None):
    #Modified from torch_geometric.data.download_url

    #Downloads the content of an URL to a specific folder.

    #Args:
        #url (str): The URL.
        #folder (str): The folder.
        #log (bool, optional): If :obj:`False`, will not print anything to the
            #console. (default: :obj:`True`)


    if filename is None:
        filename = url.rpartition("/")[2]
        filename = filename if filename[0] == "?" else filename.split("?")[0]

    path = osp.join(folder, filename)

    if osp.exists(path):  # pragma: no cover
        if log and "pytest" not in sys.modules:
            print(f"Using existing file {filename}", file=sys.stderr)
        return path

    if log and "pytest" not in sys.modules:
        print(f"Downloading {url}", file=sys.stderr)

    os.makedirs(osp.expanduser(osp.normpath(folder)), exist_ok=True)

    context = ssl._create_unverified_context()
    data = urllib.request.urlopen(url, context=context)

    with open(path, "wb") as f:
        # workaround for https://bugs.python.org/issue42853
        while True:
            chunk = data.read(10 * 1024 * 1024)
            if not chunk:
                break
            f.write(chunk)

    return path


def load_relbench_dataset(url, raw_dir):
    # Converts relbench dataset to DGL Graph format
    download_path = download_url(url, raw_dir)
    # data = np.load(download_path, allow_pickle=True)
    # data = pd.read_pickle(download_path)
    with open(download_path, 'rb') as f:
        data = pickle.load(f)

    if isinstance(data['node_features'], list) and all(isinstance(tensor, torch.Tensor) for tensor in data['node_features']):
      for tensor in data['node_features']:
        node_features = torch.stack(data['node_features'])

    if isinstance(data['labels'], list) and all(isinstance(tensor, torch.Tensor) for label in data['labels']):
      for label in data['labels']:
        labels = torch.stack(data['labels'])

    # node_features = torch.tensor(data['node_features'])
    # labels = torch.tensor(data['labels'])
    edges = torch.tensor(data['edges'])

    graph = dgl.graph((edges[:, 0], edges[:, 1]),
                      num_nodes=len(node_features), idtype=torch.int32)
    num_classes = len(labels.unique())
    train_mask, val_mask, test_mask = torch.tensor(data['train_mask']), torch.tensor(data['val_mask']), torch.tensor(
        data['test_mask'])

    return graph, labels, num_classes, node_features, train_mask, val_mask, test_mask


def load_heterophilous_dataset(url, raw_dir):
    # Wrap Heterophilous to DGL Graph Dataset format https://arxiv.org/pdf/2302.11640.pdf
    download_path = download_url(url, raw_dir)
    data = np.load(download_path)
    node_features = torch.tensor(data["node_features"])
    labels = torch.tensor(data["node_labels"])
    edges = torch.tensor(data["edges"])

    #
    '''
    print(f"node_features è un: {type(data['node_features'])} con size: {data['node_features'].shape}")
    print(data['node_features'][0])
    print(f"node_labels è un: {type(data['node_labels'])} con size: {data['node_labels'].shape}")
    print(data['node_labels'][0])
    print(f"edges è un: {type(data['edges'])} con size: {data['edges'].shape}")
    print(data['edges'][0])
    '''
    #

    graph = dgl.graph(
        (edges[:, 0], edges[:, 1]), num_nodes=len(node_features), idtype=torch.int
    )
    num_classes = len(labels.unique())
    num_targets = 1 if num_classes == 2 else num_classes
    if num_targets == 1:
        labels = labels.float()
    train_masks = torch.tensor(data["train_masks"]).T
    val_masks = torch.tensor(data["val_masks"]).T
    test_masks = torch.tensor(data["test_masks"]).T

    '''
    print(f"la size della train mask è: {data['train_masks'].shape}")
    print(data['train_masks'][0])
    print(data['train_masks'][1])
    print(data['train_masks'][2])
    '''

    return graph, labels, num_classes, node_features, train_masks, val_masks, test_masks


class CombinedDataset(pl.LightningDataModule):
    def __init__(self, train_ds_dict, eval_ds_dict, cfg):
        super().__init__()
        self.train_ds_dict = train_ds_dict
        self.eval_ds_dict = eval_ds_dict
        self.all_ds = list(self.train_ds_dict.values()) + list(
            self.eval_ds_dict.values()
        )
        self.cfg = cfg

    def to(self, device):
        for ds in self.all_ds:
            ds.to(device)

    def train_dataloader(self):
        sub_dataloaders = {
            name: ds.train_dataloader() for name, ds in self.train_ds_dict.items()
        }
        return pl.utilities.combined_loader.CombinedLoader(sub_dataloaders, "min_size")

    def val_dataloader(self):
        sub_dataloaders = {
            name: ds.val_dataloader() for name, ds in self.eval_ds_dict.items()
        }
        # Use max_size instead of max_size_cycle to avoid repeated evaluation on small datasets
        return pl.utilities.combined_loader.CombinedLoader(sub_dataloaders, "max_size")

    def test_dataloader(self):
        sub_dataloaders = {
            name: ds.test_dataloader() for name, ds in self.eval_ds_dict.items()
        }
        # Use max_size instead of max_size_cycle to avoid repeated evaluation on small datasets
        return pl.utilities.combined_loader.CombinedLoader(sub_dataloaders, "max_size")


class GraphDataset(pl.LightningDataModule):
    def __init__(
            self,
            cfg,
            ds_name,
            cache_dir,
            train_batch_size=256,
            val_test_batch_size=256,
            n_hops=1,
            preprocess_device=torch.device("cpu"),
            permute_label=False,
    ):
        super().__init__()
        self.cfg = cfg
        self.name = ds_name
        self.train_batch_size = train_batch_size
        self.permute_label = permute_label  # For checking label equivariance
        self.val_test_batch_size = val_test_batch_size
        self.preprocess_device = preprocess_device

        self.n_hops = n_hops

        self.data_source, ds_alias = cfg["_ds_meta_data"][ds_name].split(", ")
        self.gidtype = None
        self.dist = None
        self.unmasked_pred = None
        if self.data_source == "pyg":
            components = ds_alias.split(".")
            ds_init_args = {
                "_target_": f"torch_geometric.datasets.{ds_alias}",
                "root": f"{cfg.dirs.data_storage}{self.data_source}/{ds_alias}/",
            }
            if len(components) == 2:  # If sub-dataset
                ds_init_args["_target_"] = f"torch_geometric.datasets.{components[0]}"
                ds_init_args["name"] = components[1]
        elif self.data_source == "dgl":
            ds_init_args = {
                "_target_": f"dgl.data.{ds_alias}",
                "raw_dir": f"{cfg.dirs.data_storage}{self.data_source}/",
            }
        elif self.data_source == "ogb":
            ds_init_args = {
                "_target_": f"ogb.nodeproppred.DglNodePropPredDataset",
                "root": f"{cfg.dirs.data_storage}{self.data_source}/",
                "name": ds_alias,
            }
        elif self.data_source == "heterophilous":
            target = "graphany.data.load_heterophilous_dataset"
            url = f"https://raw.githubusercontent.com/yandex-research/heterophilous-graphs/main/data/{ds_alias}.npz"
            ds_init_args = {
                "_target_": target,
                "raw_dir": f"{cfg.dirs.data_storage}{self.data_source}/",
                "url": url,
            }
        elif self.data_source == "relbench":
            target = "graphany.data.load_relbench_dataset"
            url = f"https://raw.githubusercontent.com/RiccardoRomeo01/BDATM_project_public_data/main/GraphAny_datasets/{ds_alias}.pkl"
            ds_init_args = {
                "_target_": target,
                "raw_dir": f"{cfg.dirs.data_storage}{self.data_source}/",
                "url": url,
            }
        else:
            raise NotImplementedError(f"Unsupported {self.data_source=}")
        self.data_init_args = OmegaConf.create(ds_init_args)
        # self.cache_f_name = osp.join(
        #     cache_dir, f'{self.name}_{n_hops}')
        if cfg.get("feat_chn"):
            all_channels = "+".join([cfg.feat_chn, cfg.pred_chn])
            all_hops = re.findall(r"\d+", all_channels)
            n_hops = max(max([int(_) for _ in all_hops]), n_hops)

        self.split_index = 0
        (
            self.g,
            self.label,
            self.feat,
            self.train_mask,
            self.val_mask,
            self.test_mask,
            self.num_class,
        ) = self.load_dataset(self.data_init_args)
        self.n_nodes, self.n_edges = self.g.num_nodes(), self.g.num_edges()
        self.cache_f_name = osp.join(
            cache_dir,
            f"{self.name}_{n_hops}hop_selfloop={cfg.add_self_loop}_bidirected={cfg.to_bidirected}_split="
            f"{self.split_index}.pt",
        )

        self.dist_f_name = osp.join(
            cache_dir,
            f"{self.name}_{n_hops}hop_selfloop={cfg.add_self_loop}_bidirected={cfg.to_bidirected}_split="
            f"{self.split_index}_{cfg.feat_chn}_entropy={cfg.entropy}_dist.pt",
        )

        self.gidtype = self.g.idtype
        self.train_indices = self.train_mask.nonzero().view(-1)

        (
            self.features,
            self.unmasked_pred,
            self.dist,
        ) = self.prepare_prop_features_logits_and_dist_features(
            self.g, self.feat, n_hops=cfg.n_hops
        )
        # Remove the graph, as GraphAny doesn't use it in training
        del self.g
        del self.feat
        torch.cuda.empty_cache()

    def to(self, device):  # Supports nested dictionary
        def to_device(input):
            if input is None:
                return None
            elif isinstance(input, dict):
                return {key: to_device(value) for key, value in input.items()}
            elif isinstance(input, list):
                return [to_device(item) for item in input]
            elif hasattr(input, "to"):
                return input.to(device)
            else:
                return (
                    input  # Return as is if it's not a tensor or any nested structure
                )

        # Apply to_device to all attributes that may contain tensors
        attrs = [
            "label",
            "feat",
            "train_mask",
            "val_mask",
            "test_mask",
            "train_indices",
            "unmasked_pred",
        ]
        for attr in attrs:
            if hasattr(self, attr):
                setattr(self, attr, to_device(getattr(self, attr)))

    def load_dataset(self, data_init_args):
        dataset = instantiate(data_init_args)

        if self.data_source == "ogb":
            split_idx = dataset.get_idx_split()
            train_indices, valid_indices, test_indices = (
                split_idx["train"],
                split_idx["valid"],
                split_idx["test"],
            )
            # graph: dgl graph object, label: torch tensor of shape (num_nodes, num_tasks)
            g, label = dataset[0]
            label = label.view(-1)

            def to_mask(indices):
                mask = torch.BoolTensor(g.number_of_nodes()).fill_(False)
                mask[indices] = 1
                return mask

            train_mask, val_mask, test_mask = map(
                to_mask, (train_indices, valid_indices, test_indices)
            )

            num_class = label.max().item() + 1

            feat = g.ndata["feat"]
        elif self.data_source == "heterophilous":
            g, label, num_class, feat, train_mask, val_mask, test_mask = dataset
        elif self.data_source == "relbench":
            g, label, num_class, feat, train_mask, val_mask, test_mask = dataset
        elif self.data_source == "dgl":
            g = dataset[0]
            num_class = dataset.num_classes

            # get node feature
            feat = g.ndata["feat"]

            # get data split
            train_mask = g.ndata["train_mask"]
            val_mask = g.ndata["val_mask"]
            test_mask = g.ndata["test_mask"]

            label = g.ndata["label"]
        elif self.data_source == "pyg":
            g = dgl.graph((dataset.edge_index[0], dataset.edge_index[1]))
            n_nodes = dataset.x.shape[0]
            num_class = dataset.num_classes
            # get node feature
            feat = dataset.x
            label = dataset.y

            if (
                    hasattr(dataset, "train_mask")
                    and hasattr(dataset, "val_mask")
                    and hasattr(dataset, "test_mask")
            ):
                train_mask, val_mask, test_mask = (
                    dataset.train_mask,
                    dataset.val_mask,
                    dataset.test_mask,
                )
            else:
                if label.ndim > 1:
                    raise NotImplementedError(
                        "Multi-Label classification currently unsupported."
                    )
                logging.warning(
                    f"No dataset split found for {self.name}, splitting with semi-supervised settings!!"
                )
                train_mask, val_mask, test_mask = get_data_split_masks(
                    n_nodes, label, 20 * num_class, seed=self.cfg.seed
                )

                self.split_index = self.cfg.seed
        else:
            raise NotImplementedError(f"Unsupported {self.data_source=}")
        if train_mask.ndim == 1:
            pass  # only one train/val/test split
        elif train_mask.ndim == 2:
            # ! Multiple splits
            # Modified: Use the ${seed} split if not specified!
            split_index = self.data_init_args.get("split", self.cfg.seed)
            # Avoid invalid split index
            self.split_index = split_index = (split_index % train_mask.ndim)
            train_mask = train_mask[:, split_index].squeeze()
            val_mask = val_mask[:, split_index].squeeze()
            if test_mask.ndim == 2:
                test_mask = test_mask[:, split_index].squeeze()
        else:
            raise ValueError("train/val/test masks have more than 2 dimensions")
        print(
            f"{self.name} {g.num_nodes()} {g.num_edges()} {feat.shape[1]} {num_class} {len(train_mask.nonzero())}"
        )

        if self.cfg.add_self_loop:
            g = dgl.add_self_loop(g)
        else:
            g = dgl.remove_self_loop(g)
        if self.cfg.to_bidirected:
            g = dgl.to_bidirected(g)
        g = dgl.to_simple(g)  # Remove duplicate edges.
        return g, label, feat, train_mask, val_mask, test_mask, num_class

    def compute_linear_gnn_logits(
            self, features, n_per_label_examples, visible_nodes, bootstrap=False
    ):
        # Compute and save LinearGNN logits into a dict. Note the computation is on CPU as torch does not support
        # the gelss driver on GPU currently.
        preds = {}
        label, num_class, device = self.label, self.num_class, torch.device("cpu")
        label = label.to(device)
        visible_nodes = visible_nodes.to(device)
        for channel, F in features.items():
            F = F.to(device)
            if bootstrap:
                ref_nodes = sample_k_nodes_per_label(
                    label, visible_nodes, n_per_label_examples, num_class
                )
            else:
                ref_nodes = visible_nodes
            Y_L = torch.nn.functional.one_hot(label[ref_nodes], num_class).float()
            with timer(
                    f"Solving with CPU driver (N={len(ref_nodes)}, d={F.shape[1]}, k={num_class})",
                    logger.debug,
            ):
                W = torch.linalg.lstsq(
                    F[ref_nodes.cpu()].cpu(), Y_L.cpu(), driver="gelss"
                )[0]
            preds[channel] = F @ W

        return preds

    def compute_channel_logits(self, features, visible_nodes, sample, device):
        pred_logits = self.compute_linear_gnn_logits(
            {
                c: features[c]
                for c in set(self.cfg.feat_channels + self.cfg.pred_channels)
            },
            self.cfg.n_per_label_examples,
            visible_nodes,
            bootstrap=sample,
        )
        return {c: logits.to(device) for c, logits in pred_logits.items()}

    def prepare_prop_features_logits_and_dist_features(self, g, input_feats, n_hops):
        # Calculate Low-pass features containing AX, A^2X and High-pass features
        # (I-A)X, and (I-A)^2X
        if not os.path.exists(self.cache_f_name):
            g = g.to(self.preprocess_device)
            with timer(
                    f"Computing {self.name} message passing and normalized predictions to file {self.cache_f_name}",
                    logger.info,
            ):
                dim = input_feats.size(1)
                LP = torch.zeros(n_hops, g.number_of_nodes(), dim).to(
                    self.preprocess_device
                )
                HP = torch.zeros(n_hops, g.number_of_nodes(), dim).to(
                    self.preprocess_device
                )

                g.ndata["LP"] = input_feats.to(self.preprocess_device)
                g.ndata["HP"] = input_feats.to(self.preprocess_device)
                for hop_idx in range(n_hops):
                    # D^-1 A filter
                    g.update_all(fn.copy_u("LP", "temp"), fn.mean("temp", "LP"))

                    # (I - D^-1A) filter
                    g.update_all(fn.copy_u("HP", "temp"), fn.mean("temp", "HP_out"))
                    g.ndata["HP"] = g.ndata["HP"] - g.ndata["HP_out"]

                    LP[hop_idx] = g.ndata["LP"].clone()
                    HP[hop_idx] = g.ndata["HP"].clone()
                lp_feat_dict = {f"L{l + 1}": x for l, x in enumerate(LP)}
                hp_feat_dict = {f"H{l + 1}": x for l, x in enumerate(HP)}

                features = {"X": input_feats, **lp_feat_dict, **hp_feat_dict}
                unmasked_pred = self.compute_channel_logits(
                    features,
                    self.train_indices,
                    sample=False,
                    device=self.preprocess_device,
                )
                torch.save((features, unmasked_pred), self.cache_f_name)
        else:
            features, unmasked_pred = torch.load(self.cache_f_name, map_location="cpu")
        if not os.path.exists(self.dist_f_name):
            with timer(
                    f"Computing {self.name} conditional gaussian distances "
                    f"and save to {self.dist_f_name}",
                    logger.info,
            ):
                # y_feat: n_nodes, n_channels, n_labels
                y_feat = np.stack(
                    [unmasked_pred[c].cpu().numpy() for c in self.cfg.feat_channels],
                    axis=1,
                )
                # Conditional gaussian probability
                bsz, n_channel, n_class = y_feat.shape
                dist_feat_dim = n_channel * (n_channel - 1)
                # Conditional gaussian probability
                cond_gaussian_prob = np.zeros((bsz, n_channel, n_channel))
                for i in range(bsz):
                    cond_gaussian_prob[i, :, :] = get_entropy_normed_cond_gaussian_prob(
                        y_feat[i, :, :], self.cfg.entropy
                    )
                dist = np.zeros((bsz, dist_feat_dim), dtype=np.float32)

                # Compute pairwise distances between channels n_channels(n_channels-1)/2 total features
                pair_index = 0
                for c in range(n_channel):
                    for c_prime in range(n_channel):
                        if c != c_prime:  # Diagonal distances are useless
                            dist[:, pair_index] = cond_gaussian_prob[:, c, c_prime]
                            pair_index += 1

                dist = torch.from_numpy(dist)
                torch.save(dist, self.dist_f_name)
        else:
            dist = torch.load(self.dist_f_name, map_location="cpu")
        return features, unmasked_pred, dist

    def train_dataloader(self):
        return DataLoader(
            self.train_mask.nonzero().view(-1),
            batch_size=self.train_batch_size,
            shuffle=True,
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_mask.nonzero().view(-1), batch_size=self.val_test_batch_size
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_mask.nonzero().view(-1), batch_size=self.val_test_batch_size
        )

"""

In [39]:
path_name = 'GraphAny/graphany/data.py'
with open(path_name, 'w') as file:
    file.write(new_code)

# Testing GraphAny on F1 Dataset

In [40]:
script_path = "GraphAny/graphany/run.py"
# dataset = "F1Debug" # we want to use the F1 dataset
dataset = "Debug"
steps = 0 # we want to perform zero-shot, thus we impose zero training epochs
checkpoint_path = "GraphAny/checkpoints/graph_any_wisconsin.pt"

In [41]:
os.environ['HYDRA_FULL_ERROR'] = '1'
!source activate graphany && python {script_path} prev_ckpt={checkpoint_path} dataset={dataset} total_steps={steps}

[09:20:36] INFO     Logger initialized.                                                                                                                 logging.py:53
           INFO     Local_rank=0, working_dir=/content/temp/working_dir/Feb20-9:20-fd97f788/                                                         experiment.py:56
Done loading data from cached files.
Wisconsin 251 515 1703 5 120
Using existing file texas_4_classes.npz
node_features è un: <class 'numpy.ndarray'> con size: (183, 1703)
[0. 0. 0. ... 0. 0. 0.]
node_labels è un: <class 'numpy.ndarray'> con size: (183,)
3
edges è un: <class 'numpy.ndarray'> con size: (279, 2)
[ 0 58]
la size della train mask è: (10, 183)
[ True False  True False  True False False False  True False False  True
 False  True False  True False False False  True False  True False False
 False False  True False False False False False  True False False  True
 False  True False  True False  True False False  True  True  True False
 False False  True